### This script output a big list containing syllable intervals: [ [{syllable:[interval]}, {}, {},...  ], [sentence 2], [sentence 3], [], [], .... ]

In [2]:
import os
from scipy.io import wavfile
from dSPEECH.config import *
import json

In [3]:
sid=2
folder=meta_dir+'SEEG'+str(sid)+'/forced_alignment/' #alignment/'
filename=folder+'/phone_list_onoff_beat.npy'
phones_all=np.load(filename,allow_pickle=True)

In [4]:
# group phones into syllables, and get syllable period (start, end).

# The duration can be syllable_start+pohone_duration+phone_duration+...; 
on_beats=[]
off_beats=[]
# or it can be calculated as difference between syllable starting points;
# The syllable duration is usually longer than the previous method
on_beats2=[]
off_beats2=[]

on_beats_phones=[]
off_beats_phones=[]
oov_num=0
exclude=['oov','not-found-in-aud']
for i, phones_sentence in enumerate(phones_all):
    # log the text phone/syllable
    on_beats_sentence=[]
    off_beats_sentence=[]
    on_beats_syllable=[]
    off_beats_syllable=[]
    # syllable intervals within a sentence
    on_beats_sentenceT=[]
    off_beats_sentenceT=[]
    # syllable intervals within a sentence, calculated as difference between syllable starts
    on_beats_sentenceT2=[]
    off_beats_sentenceT2=[]
    for j,phone_dict in enumerate(phones_sentence):
        key=list(phone_dict.keys())[0]
        value=phone_dict[key]
        phone_start=value[0]
        phone_duration=value[1]
        beat=value[-1]
        if key=='oov':
            oov_num=oov_num+1
        #elif key=='not-found-in-aud':
        #    print('haha')
        #else: 
        if j==0:
            pre_beat=-1
            pre_key=key
            pre_syllable_start=phone_start
            pre_syllable_end=pre_syllable_start+phone_duration
            #off_beats_sentence.append(pre_key)
            #off_beats_syllable.append(pre_key)
        else:
            if beat!=pre_beat: # start a new syllable
                if pre_beat==-1: # fetch last the phone of previous syllable
                    off_beats_syllable.append(pre_key)
                    off_beats_sentence.append(off_beats_syllable)
                else:
                    on_beats_syllable.append(pre_key)
                    on_beats_sentence.append(on_beats_syllable)

                if pre_beat==-1:
                    #off_beats_sentence.append(pre_key)
                    if pre_key not in exclude: off_beats_sentenceT.append(
                        {''.join([i for i in off_beats_syllable]):[pre_syllable_start,pre_syllable_end]}) #off_beats
                    # difference between syllable starts
                    if pre_key not in exclude: off_beats_sentenceT2.append(
                        {''.join([i for i in off_beats_syllable]):[pre_syllable_start,phone_start]}) 
                    off_beats_syllable=[]
                    #off_beats_syllable.append(pre_key)
                elif pre_beat==1:
                    #on_beats_sentence.append(pre_key)
                    if pre_key not in exclude:on_beats_sentenceT.append(
                        {''.join([i for i in on_beats_syllable]):[pre_syllable_start,pre_syllable_end]})
                    if pre_key not in exclude:on_beats_sentenceT2.append(
                        {''.join([i for i in on_beats_syllable]):[pre_syllable_start,phone_start]}) 
                    on_beats_syllable=[]
                    #on_beats_syllable.append(pre_key)
                pre_syllable_start=phone_start
                pre_syllable_end=pre_syllable_start+phone_duration
                pre_key=key
            else: # still within the same syllable
                pre_syllable_end=phone_start+phone_duration
                if pre_beat==-1:
                    off_beats_syllable.append(pre_key)
                    #off_beats_sentence.append(pre_key)
                elif pre_beat==1:
                    on_beats_syllable.append(pre_key)
                    #on_beats_sentence.append(pre_key)
                pre_key=key
                if j==len(phones_sentence)-1: # the last phone
                    if pre_beat==-1:
                        off_beats_syllable.append(pre_key)
                        off_beats_sentence.append(off_beats_syllable)
                        if pre_key not in exclude:off_beats_sentenceT.append(
                            {''.join([i for i in off_beats_syllable]):[pre_syllable_start,pre_syllable_end]}) #off_beats
                        if pre_key not in exclude:off_beats_sentenceT2.append(
                            {''.join([i for i in off_beats_syllable]):[pre_syllable_start,pre_syllable_end]})
                    else:
                        on_beats_syllable.append(pre_key)
                        on_beats_sentence.append(on_beats_syllable)
                        if pre_key not in exclude:on_beats_sentenceT.append(
                            {''.join([i for i in on_beats_syllable]):[pre_syllable_start,pre_syllable_end]})
                        if pre_key not in exclude:on_beats_sentenceT2.append(
                            {''.join([i for i in on_beats_syllable]):[pre_syllable_start,pre_syllable_end]}) 
        pre_beat=beat
    on_beats.append(on_beats_sentenceT)
    off_beats.append(off_beats_sentenceT)
    on_beats2.append(on_beats_sentenceT2)
    off_beats2.append(off_beats_sentenceT2)
    on_beats_phones.append(on_beats_sentence) # pure text, not time info, debug usage.
    off_beats_phones.append(off_beats_sentence)
print('OOV numebr: '+str(oov_num)+'.')

OOV numebr: 145.


In [5]:
off_beats2[7] # [0,0] means it's a not-found-in-audio syllable;

[{'ah': [0.0, 1.05]},
 {'ihz': [1.84, 2.11]},
 {'wihth': [3.11, 3.15]},
 {'aor': [4.24, 4.49]}]

### Looks like some on_beats were missing: there are 5 not-found-in-audio syllables

370+381+5=756<798(800-2nans); still missing some syllables: these missing syllable are from the oovs: 1 syllables contains approximately 2-3 oovs;

To investigate, you can compare the combination of on_beats_phones and off_beats_phones with the excel file.

Should not be a big deal, not worth the time.

In [6]:
on_beats2_tmp=[[list(syl.values())[0] for syl in sentence] for sentence in on_beats2]
off_beats2_tmp=[[list(syl.values())[0] for syl in sentence] for sentence in off_beats2]

In [7]:
len([i for sentence in on_beats2_tmp for syl in sentence])

366

In [8]:
len([i for sentence in off_beats2_tmp for syl in sentence])

380

### Deal with the oov; there are too many.

### Save for next step: on/off beat classification.

In [9]:
result={}
result['on_beats']=on_beats
result['off_beats']=off_beats
result['on_beats2']=on_beats2
result['off_beats2']=off_beats2
result['on_beats_phones']=on_beats_phones
result['off_beats_phones']=off_beats_phones

filename=folder+'/syllable_time.npy'
np.save(filename, result)